This notebook creates the binary novelty measure for a dataframe. A patent is considered novel if ANY of its backwards class citation combinations is novel. A combination is considered novel if it has never been seen for its listed focal patent's priority year


In [1]:
import pandas as pd
import numpy as np

In [2]:
def first_occurence_index(df, cite_col_name, focal_col_name):
    '''
    This function marks the first occurence of class pair. It does NOT address simultanious novelty,
    which will be addressed in a separate function. The input dataframe should already by sorted by date,
    that way we can simply use the index to decide the first time a pair occurs
    
    methodology
    1. select the citation and focal class columns
    2. make classes invariant to citation order, e.g. class A citing class B is equal to class B citing class A 
    3. Only keep the first occurance of class pair by dropping the duplicates
    4. Take the index of these first occurances
    '''
    indx = df[[cite_col_name,focal_col_name]].apply(sorted, axis = 1, result_type='expand')\
    .drop_duplicates(keep='first').index
    
    return indx

In [3]:
def simultanious_occurence_index(df, cite_col_name, focal_col_name):
    '''
    This function marks patents of simultanious novelty in a given year.
    If two separate patents use the same class pair in a year, and that year was occurence
    of that pairing, then both patents should be listed as novel.
    The first occurences should be identified prior to using this function
    
    methodology
    1. select the citation and focal class columns
    2. make classes invariant to citation order, e.g. class A citing class B is equal to class B citing class A 
    3. Create an intermediate dataframe with the original class columns and the invariant class columns
    4. from the intermediate select the rows that have been marked as novel and save their unique
    year-class1-class2 combination
    5. select the rows in intermediate that also match any of the year-class1-class2 from the previous step
    6. return the indices of these rows
    '''
    df1 = df[[cite_col_name,focal_col_name]].apply(sorted, axis=1, result_type='expand')\
    .rename({0:'sorted_class1', 1:'sorted_class2'}, axis='columns') 
    df1 = df1.reset_index(drop=True)
    
    #acreate intermediate
    intermediate = pd.concat([df, df1], axis=1, join_axes=[df1.index])
    cols = ['priority_date','sorted_class1','sorted_class2']
    seen = intermediate.loc[intermediate['first_seen'] == 1][cols]

    # converting datafrane into series of strings for easy matching that does not rely on the index
    a = intermediate[cols].astype(str).sum(1)
    b = seen.astype(str).sum(1)
    also_seen = intermediate.loc[a.isin(b)].index
    
    return also_seen

In [4]:
def mark_binary_novelty(df, cite_col_name, focal_col_name):
    '''
    This identifies novel patents using the binary measure
    
    methodology
    1. Mark first novel class pairs
    2. Mark simultaniously novel class pairs
    '''
    
    df['first_seen'] = 0
    
    #standarize datatype of citations to string
    df[cite_col_name] = df[cite_col_name].astype(str)
    
    # Exploit indices by sorting by date
    df = df.sort_values('priority_date').reset_index()
    indx = first_occurence_index(df, cite_col_name, focal_col_name)
    df.loc[indx, 'first_seen'] = 1
    
    #reset the index again
    df.reset_index(drop=True, inplace=True)
    
    #find simultanious novelty and mark them
    also_seen = simultanious_occurence_index(df, cite_col_name, focal_col_name)
    df.loc[also_seen, 'first_seen'] = 1
    return df


In [5]:
'''
import design citing design mainclass
design citing utility mainclass
design citing design subclass
design citing utility subclass
'''
# main_d2d = pd.read_csv('data/final_main_d2d.csv')
# main_d2u = pd.read_csv('data/final_main_d2u.csv')
# sub_d2d = pd.read_csv('data/final_sub_d2d.csv')
# sub_d2u = pd.read_csv('data/final_sub_d2u.csv')

main_d2d = pd.read_csv('data/final_main_d2d_drops.csv')
main_d2u = pd.read_csv('data/final_main_d2u_drops.csv')
sub_d2d = pd.read_csv('data/final_sub_d2d_drops.csv')
sub_d2u = pd.read_csv('data/final_sub_d2u_drops.csv')


In [6]:
def sort_reset(df):
    return df.sort_values(['priority_date','patent_number']).reset_index(drop=True)

In [7]:
main_d2d = sort_reset(main_d2d)
main_d2u = sort_reset(main_d2u)
sub_d2d = sort_reset(sub_d2d)
sub_d2u = sort_reset(sub_d2u)

Create the measures from the data

In [8]:
main_d2d = mark_binary_novelty(main_d2d,'cite_mainclass','focal_mainclass')

#examin first 50
main_d2d.head(50).sort_values(['priority_date','cite_mainclass','focal_mainclass'])

,index,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen
0,0,D523461,1906,D14,D16,1
28,942,D268558,1980,D06,D07,1
5,922,D268500,1980,D06,D08,1
7,920,D268481,1980,D06,D09,1
4,923,D268500,1980,D06,D15,1
45,882,D268285,1980,D06,D24,1
3,924,D268500,1980,D06,D30,1
6,921,D268484,1980,D07,D09,1
15,912,D268444,1980,D07,D11,1
48,879,D268255,1980,D07,D11,1


In [9]:
main_d2u = mark_binary_novelty(main_d2u,'cite_mainclass','focal_mainclass')

#examin first 50
main_d2u.head(50).sort_values(['priority_date','cite_mainclass','focal_mainclass'])

,index,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen
6,6,D523461,1906,16,D16,1
7,7,D523461,1906,2,D16,1
3,3,D523461,1906,345,D16,1
4,4,D523461,1906,348,D16,1
2,2,D523461,1906,351,D16,1
5,5,D523461,1906,359,D16,1
8,8,D523461,1906,379,D16,1
0,0,D523461,1906,381,D16,1
1,1,D523461,1906,455,D16,1
21,2948,D268935,1980,106,D19,1


In [10]:
sub_d2d = mark_binary_novelty(sub_d2d,'cite_subclass','focal_subclass')

#examin first 50
sub_d2d.head(50).sort_values(['priority_date','cite_subclass','focal_subclass'])

,index,patent_number,priority_date,cite_subclass,focal_subclass,first_seen
16,3,D523461,1906,D14/189,D16/309,1
17,4,D523461,1906,D14/189,D16/330,1
23,5,D523461,1906,D14/189,D16/335,1
12,21,D523461,1906,D14/206,D16/309,1
2,22,D523461,1906,D14/206,D16/330,1
1,23,D523461,1906,D14/206,D16/335,1
5,18,D523461,1906,D14/223,D16/309,1
4,19,D523461,1906,D14/223,D16/330,1
3,20,D523461,1906,D14/223,D16/335,1
21,8,D523461,1906,D16/306,D16/309,1


In [11]:
sub_d2u = mark_binary_novelty(sub_d2u,'cite_subclass','focal_subclass')

#examin first 50
sub_d2u.head(50).sort_values(['priority_date','cite_subclass','focal_subclass'])

,index,patent_number,priority_date,cite_subclass,focal_subclass,first_seen
21,85,D523461,1906,16/228,D16/330,1
22,86,D523461,1906,16/228,D16/335,1
47,153,D523461,1906,2/171.3,D16/309,1
48,154,D523461,1906,2/171.3,D16/330,1
49,155,D523461,1906,2/171.3,D16/335,1
15,120,D523461,1906,2/422,D16/309,1
16,121,D523461,1906,2/422,D16/330,1
38,122,D523461,1906,2/422,D16/335,1
39,123,D523461,1906,2/423,D16/309,1
40,124,D523461,1906,2/423,D16/330,1


In [12]:
def is_novel_patent_level(df):
    '''
    A patent is considered novel is any of it's class combinations is considered novel
    
    Methodology:
    1. group patents by their patent number
    2. if any of the pairs are novel, mark the patent as novel
    '''
    
    is_novel = df.groupby('patent_number')\
    .apply(lambda x: np.any(x.first_seen)).astype(int)\
    .reset_index().rename(index=int, columns={0:'is novel'})
    return is_novel

In [13]:
def novel_count_patent_level(df):
    '''
    This function counts the number of binary novel combinations in a patent
    
    Methodolgy:
    1. group patents by their patent number
    2. count the number of novel combinations in each patent
    '''
    
    novel_count = df.groupby('patent_number')\
    .apply(lambda x: np.sum(x.first_seen))\
    .reset_index().rename(index= int, columns={0:'novelty count'})
    return novel_count

In [14]:
def consolidate(df):
    '''
    This function creates a dataframe that consolidates the
    novely count and is novelty measure at the patent level
    '''
    
    is_novel = is_novel_patent_level(df)
    count = novel_count_patent_level(df)
    
    out = pd.merge(is_novel, count)
    return out
    

Consolidate the data at the patent level

In [15]:
main_d2d_out = consolidate(main_d2d)
main_d2u_out = consolidate(main_d2u)
sub_d2d_out = consolidate(sub_d2d)
sub_d2u_out = consolidate(sub_d2u)

In [16]:
sub_d2d_out.head()

,patent_number,is novel,novelty count
0,D258990,1,2
1,D259247,1,8
2,D259248,1,8
3,D259249,1,10
4,D259250,1,10


In [17]:
# Save the resulting data frames for furher analysis
# main_d2d_out.to_csv('data/d2d_main_binary.csv', index_label = False)
# main_d2u_out.to_csv('data/d2u_main_binary.csv', index_label = False)
# sub_d2d_out.to_csv('data/d2d_sub_binary.csv', index_label = False) 
# sub_d2u_out.to_csv('data/d2u_sub_binary.csv', index_label = False)

In [18]:
main_d2d_out.to_csv('data/d2d_main_binary_drops.csv', index_label = False)
main_d2u_out.to_csv('data/d2u_main_binary_drops.csv', index_label = False)
sub_d2d_out.to_csv('data/d2d_sub_binary_drops.csv', index_label = False) 
sub_d2u_out.to_csv('data/d2u_sub_binary_drops.csv', index_label = False)